### Dataset used:  
https://www.kaggle.com/dansbecker/5-celebrity-faces-dataset/downloads/5-celebrity-faces-dataset.zip/3

### Download Pre-trained model:
https://drive.google.com/open?id=1pwQ3H4aJ8a6yyJHZkTwtjcL4wYWQb7bn

To use the code with your photos, place the photos of a person(~5-6 should be enough) in a folder having the name of the person. 

In [1]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
    return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for person: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

# load train dataset
dir_loc = input("Enter the path of the directory with training data: ")
dir_loc.replace('\\', '/')
dir_loc += '/'
trainX, trainy = load_dataset(dir_loc)
print(trainX.shape, trainy.shape)
# load test dataset
dir_loc = input("Enter the path of the directory with testing data: ")
dir_loc.replace('\\', '/')
dir_loc += '/'
testX, testy = load_dataset(dir_loc)
# save arrays to one file in compressed format
savez_compressed('init_face_array.npz', trainX, trainy, testX, testy)

Enter the path of the directory with training data: C:\Users\Sarthak\Desktop\Practice\FaceNet\train
>loaded 14 examples for person: ben_afflek
>loaded 17 examples for person: elton_john
>loaded 21 examples for person: jerry_seinfeld
>loaded 19 examples for person: madonna
>loaded 22 examples for person: mindy_kaling
(93, 160, 160, 3) (93,)
Enter the path of the directory with testing data: C:\Users\Sarthak\Desktop\Practice\FaceNet\val
>loaded 5 examples for person: ben_afflek
>loaded 5 examples for person: elton_john
>loaded 5 examples for person: jerry_seinfeld
>loaded 5 examples for person: madonna
>loaded 5 examples for person: mindy_kaling


In [2]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
#from numpy import asarray
#from numpy import savez_compressed
from keras.models import load_model

# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

# load the face dataset
#data = load('init_face_array.npz')
#trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
model = load_model('fa.h5')  #read readme file. 
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

Using TensorFlow backend.


Loaded:  (93, 160, 160, 3) (93,) (25, 160, 160, 3) (25,)


F:\Anaconda\envs\tfenv\lib\site-packages\keras\engine\saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Loaded Model
(93, 128)
(25, 128)


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from random import choice
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load dataset
data = load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
testX_faces = data['arr_2']
# test model on a random example from the test dataset
selection = choice([i for i in range(testX.shape[0])])
#selection = 5
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])


#print the image with the predicted and expected output, please fork and help
'''print(random_face_pixels.shape)
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()'''

Dataset: train=93, test=25
Accuracy: train=100.000, test=100.000
Predicted: mindy_kaling (92.792)
Expected: mindy_kaling


F:\Anaconda\envs\tfenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\envs\tfenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


"print(random_face_pixels.shape)\npyplot.imshow(random_face_pixels)\ntitle = '%s (%.3f)' % (predict_names[0], class_probability)\npyplot.title(title)\npyplot.show()"

# Giving input from our side (From outside the dataset):

In [ ]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array


In [ ]:
#take input an image, find it's face embedding and then get prediction:
from os import listdir
from os.path import isdir

img_path = input("Enter the path of the image: ")
img_path.replace('\\', '/')
img_path = img_path + '/'
if not isdir(img_path):
    print('\t\t***ERROR-ENTER CORRECT PATH***')
else:
    faces = list()
    # enumerate files
    for filename in listdir(img_path):
        # path
        path = img_path + filename
        # extract_face function will return the face after mtcnn
        face = extract_face(path)
        
        

In [ ]:
#face has been extracted, now moving on to embeddings
face.shape

In [ ]:
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [ ]:
from keras.models import load_model
model_nn = load_model('fa.h5')

In [58]:
from numpy import expand_dims
t_emb = get_embedding(model_nn, face)

In [ ]:
t_emb.shape

In [ ]:
# prediction for the face
samples = expand_dims(t_emb, axis=0)
print(samples.shape)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
#print('Expected: %s' % random_face_name[0])

In [63]:
yhat_class

array([1], dtype=int64)

In [64]:
yhat_prob

array([[3.0000009e-14, 1.0000000e+00]])